In [1]:
# ----------------- SPECIFY FILE LOCATIONS HERE -----------------

# S3 paths
s3_bucket = 'sdg-project'
model_name = 'bal_model1'
artifact_name = 'pytorch-training-2023-10-26-04-05-28-024'

# Model location path
s3_model_path = f's3://{s3_bucket}/models/{model_name}/artifacts/{artifact_name}/output/model.tar.gz'

# Original test data path
#test_data_path = 'data/old_data/test_course.csv'
test_data_path = 'data/bal_test.csv'
s3_test_data_path = f's3://{s3_bucket}/{test_data_path}'

# Transformed (text only) test data path
transformed_data_path = 'data/test_split/text_only_test.csv'
s3_transformed_data_path = f's3://{s3_bucket}/{transformed_data_path}'

# Prediction output path
prediction_output_path = f's3://{s3_bucket}/models/{model_name}/predictions/'

# Local paths
local_tarball_path = 'model.tar.gz'
local_test_data_path = 'Split_Data/text_only_test.csv'
local_results_path = 'batch_results.csv'


In [5]:
!pip install transformers==4.18.0

import boto3
import sagemaker
import torch
import os
from sagemaker import get_execution_role
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
import numpy as np
from sagemaker.pytorch import PyTorchModel
import transformers
import tarfile

role = get_execution_role()
sagemaker_session = sagemaker.Session()
bucket = 'sdg-project'

print(transformers.__version__)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
4.18.0


In [6]:
print(s3_model_path)


s3://sdg-project/models/bal_model1/artifacts/pytorch-training-2023-10-26-04-05-28-024/output/model.tar.gz


In [7]:
# Download the tarball from S3
s3 = boto3.client('s3')
s3.download_file(s3_bucket, s3_model_path.replace(f's3://{s3_bucket}/', ''), local_tarball_path)

# Extract the tarball
with tarfile.open(local_tarball_path, 'r:gz') as tar:
    tar.extractall()

In [8]:
# Read the test data
test_data = pd.read_csv(f's3://{s3_bucket}/{test_data_path}')
valid_rows = test_data['Text'].apply(lambda x: isinstance(x, str))
test_data = test_data[valid_rows]

os.makedirs('Split_Data', exist_ok=True)


In [9]:
print(len(test_data))

500


In [10]:
# Check 1: Verify if any row in the 'Text' column contains unexpected double quotes
malformed_rows = test_data['Text'].str.count('"') % 2 != 0
if malformed_rows.sum() > 0:
    print(f"Warning: Found {malformed_rows.sum()} rows with unmatched double quotes!")

# Check 2: Verify if any row in the 'Text' column contains newlines 
newline_rows = test_data['Text'].str.contains('\n')
if newline_rows.sum() > 0:
    print(f"Warning: Found {newline_rows.sum()} rows with newlines!")

# Check 3: Verify if any row in the 'Text' column exceeds a certain length
# (Useful check if there's a MaxPayloadInMB constraint as mentioned in your logs)
max_length = 5 * 10**6  # e.g., 5MB
long_rows = test_data['Text'].str.len() > max_length
if long_rows.sum() > 0:
    print(f"Warning: Found {long_rows.sum()} rows exceeding {max_length} characters!")


In [11]:
# Handle unmatched double quotes
# Option 1: Remove rows with unmatched double quotes
test_data = test_data[~malformed_rows]


# Handle newlines
# Replace newlines with spaces
test_data['Text'] = test_data['Text'].str.replace('\n', ' ')


In [27]:
print(len(test_data))
true_labels = test_data.drop(columns=['Text'])
true_labels

498


,SDG 1,SDG 2,SDG 3,SDG 4,SDG 5,SDG 6,SDG 7,SDG 8,SDG 9,SDG 10,SDG 11,SDG 12,SDG 13,SDG 14,SDG 15,SDG 16,SDG 17
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,1,0,1,0,1,0,0,0,0,1,0,1,1,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
496,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
497,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
498,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [13]:
# Create a version with only the 'Text' column and save it
test_data[["Text"]].to_csv(local_test_data_path, index=False, header=False)

In [165]:
# # Load the CSV
# test_data = pd.read_csv(s3_transformed_data_path)

# # Display the first few rows
# #print(test_data.head())
# with open(local_test_data_path, 'r') as file:
#     content = file.read()
#     if content.count('"') % 2 != 0:
#         print("Mismatched quotes found!")
#     else:
#         print("Quotes seem to be okay.")

# # Check for empty or NaN rows
# empty_rows = test_data[test_data.isnull().any(axis=1)]
# if not empty_rows.empty:
#     print("Found empty or NaN rows:")
#     print(empty_rows)
# else:
#     print("No empty or NaN rows found.")

# # with open(local_test_data_path, 'r') as file:
# #     content = file.read()
# #     line_endings = set(content.splitlines(True))
# #     print(f"Line endings used: {line_endings}")

# # Check if all rows have the same number of columns
# column_counts = test_data.apply(lambda x: x.count(), axis=1)
# if len(column_counts.unique()) > 1:
#     print("Rows with varying number of columns detected!")
# else:
#     print("All rows have the same number of columns.")

In [14]:
# Upload the text-only data to S3
s3_resource = boto3.resource('s3')
s3_resource.meta.client.upload_file(local_test_data_path, s3_bucket, transformed_data_path)

!tar -tzvf model.tar.gz

tar: Ignoring unknown extended header keyword `LIBARCHIVE.creationtime'
-rw-r--r-- root/root  98841817 2023-10-26 11:26 model.pth


In [15]:
# Assuming 'model.pth' is in the current directory
with tarfile.open('model.tar.gz', 'w:gz') as tar:
    tar.add('model.pth')
    tar.add('requirements.txt')


In [16]:
import os
import tarfile

# Extract the tarball to a temp directory
temp_dir = 'temp_model_dir'
os.makedirs(temp_dir, exist_ok=True)

with tarfile.open(local_tarball_path, 'r:gz') as tar:
    tar.extractall(path=temp_dir)

# List contents of the extracted directory
extracted_files = os.listdir(temp_dir)
print("Extracted files:", extracted_files)

# Define the necessary files that should be present
required_files = ['model.pth', 'requirements.txt']

# Check if the necessary files are present
all_files_present = all(file in extracted_files for file in required_files)

if all_files_present:
    print("All necessary files are present in the model.tar.gz.")
else:
    missing_files = [file for file in required_files if file not in extracted_files]
    print(f"Missing files in model.tar.gz: {missing_files}")
    # Raise an error or handle accordingly
    raise RuntimeError(f"model.tar.gz is missing the following files: {missing_files}")

# Optionally, remove the temporary directory after verification
import shutil
shutil.rmtree(temp_dir)

Extracted files: ['model.pth', 'requirements.txt']
All necessary files are present in the model.tar.gz.


In [17]:
model_location = 'model.tar.gz' 
model = PyTorchModel(model_data=model_location, 
                     role=role,
                     framework_version='1.8.0',
                     py_version='py3',
                     entry_point='train.py',
                     sagemaker_session=sagemaker_session)

In [18]:
transformer = model.transformer(instance_count=1,
                                instance_type='ml.m5.xlarge',
                                strategy='SingleRecord',
                                assemble_with='Line',
                                output_path=prediction_output_path)

In [19]:
transformer.transform(s3_transformed_data_path, content_type='text/csv', split_type='Line')
transformer.wait()

INFO:sagemaker:Creating transform job with name: pytorch-inference-2023-10-26-13-31-37-400


...............................2023-10-26T13:36:47.914:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=6, BatchStrategy=SINGLE_RECORD
2023-10-26 13:36:45,343 [INFO ] main org.pytorch.serve.ModelServer - 
Torchserve version: 0.3.0
TS Home: /opt/conda/lib/python3.6/site-packages
Current directory: /
Temp directory: /home/model-server/tmp
Number of GPUs: 0
Number of CPUs: 4
Max heap size: 2956 M
Python executable: /opt/conda/bin/python3.6
Config file: /etc/sagemaker-ts.properties
Inference address: http://0.0.0.0:8080
Management address: http://0.0.0.0:8080
2023-10-26 13:36:45,343 [INFO ] main org.pytorch.serve.ModelServer - 
Torchserve version: 0.3.0
TS Home: /opt/conda/lib/python3.6/site-packages
Current directory: /
Temp directory: /home/model-server/tmp
Number of GPUs: 0
Number of CPUs: 4
Max heap size: 2956 M
Python executable: /opt/conda/bin/python3.6
Config file: /etc/sagemaker-ts.properties
Inference address: http://0.0.0.0:8080
Management address: http://0.0.0.0:8080


2023-10-26 13:36:59,035 [INFO ] W-9003-model_1-stdout org.pytorch.serve.wlm.WorkerLifeCycle -   Building wheel for sacremoses (setup.py): finished with status 'done'
2023-10-26 13:36:59,337 [INFO ] W-9002-model_1-stdout org.pytorch.serve.wlm.WorkerLifeCycle - Installing collected packages: zipp, importlib-metadata, regex, packaging, filelock, click, tokenizers, sacremoses, huggingface-hub, transformers
2023-10-26 13:36:59,483 [INFO ] W-9000-model_1-stdout org.pytorch.serve.wlm.WorkerLifeCycle -   Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895254 sha256=d106e57678cf77c2a37972608e7f18a43840470af87efa7dae30b78b444f2373
2023-10-26 13:36:59,483 [INFO ] W-9000-model_1-stdout org.pytorch.serve.wlm.WorkerLifeCycle -   Stored in directory: /root/.cache/pip/wheels/4c/64/31/e9900a234b23fb3e9dc565d6114a9d6ff84a72dbdd356502b4
2023-10-26 13:36:59,487 [INFO ] W-9000-model_1-stdout org.pytorch.serve.wlm.WorkerLifeCycle - Successfully built sacremoses
2023-10-26 13:3

2023-10-26 13:37:09,648 [INFO ] W-9003-model_1-stdout org.pytorch.serve.wlm.WorkerLifeCycle - Requirement already satisfied: wheel>=0.26 in /opt/conda/lib/python3.6/site-packages (from tensorboard) (0.36.2)
2023-10-26 13:37:09,648 [INFO ] W-9003-model_1-stdout org.pytorch.serve.wlm.WorkerLifeCycle - Requirement already satisfied: wheel>=0.26 in /opt/conda/lib/python3.6/site-packages (from tensorboard) (0.36.2)
2023-10-26 13:37:09,938 [INFO ] W-9000-model_1-stdout org.pytorch.serve.wlm.WorkerLifeCycle - Collecting grpcio>=1.24.3
2023-10-26 13:37:09,955 [INFO ] W-9000-model_1-stdout org.pytorch.serve.wlm.WorkerLifeCycle -   Using cached grpcio-1.48.2-cp36-cp36m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.6 MB)
2023-10-26 13:37:10,122 [INFO ] W-9000-model_1-stdout org.pytorch.serve.wlm.WorkerLifeCycle - Collecting markdown>=2.6.8
2023-10-26 13:37:10,125 [INFO ] W-9000-model_1-stdout org.pytorch.serve.wlm.WorkerLifeCycle -   Using cached Markdown-3.3.7-py3-none-any.whl (97 kB)
2023-1

2023-10-26 13:37:19,132 [WARN ] W-9003-model_1-stderr org.pytorch.serve.wlm.WorkerLifeCycle - Downloading:   0%|          | 0.00/140M [00:00<?, ?B/s]
2023-10-26 13:37:19,236 [WARN ] W-9003-model_1-stderr org.pytorch.serve.wlm.WorkerLifeCycle - Downloading:   0%|          | 35.0k/140M [00:00<07:25, 329kB/s]
2023-10-26 13:37:19,342 [WARN ] W-9003-model_1-stderr org.pytorch.serve.wlm.WorkerLifeCycle - Downloading:   0%|          | 93.0k/140M [00:00<05:12, 470kB/s]
2023-10-26 13:37:19,443 [WARN ] W-9003-model_1-stderr org.pytorch.serve.wlm.WorkerLifeCycle - Downloading:   0%|          | 221k/140M [00:00<02:58, 821kB/s] 
2023-10-26 13:37:19,544 [WARN ] W-9003-model_1-stderr org.pytorch.serve.wlm.WorkerLifeCycle - Downloading:   0%|          | 429k/140M [00:00<01:51, 1.32MB/s]
2023-10-26 13:37:19,644 [WARN ] W-9003-model_1-stderr org.pytorch.serve.wlm.WorkerLifeCycle - Downloading:   1%|          | 857k/140M [00:00<01:01, 2.39MB/s]
2023-10-26 13:37:19,132 [WARN ] W-9003-model_1-stderr org.py

...

2023-10-26 13:50:13,907 [INFO ] W-9003-model_1 org.pytorch.serve.wlm.WorkerThread - Backend response time: 1338
2023-10-26 13:50:13,907 [INFO ] W-9003-model_1-stdout MODEL_METRICS - PredictionTime.Milliseconds:1337.31|#ModelName:model,Level:Model|#hostname:c449b28b682c,requestID:11c21136-ed57-4d0a-bbfe-00292654bfc7,timestamp:1698328213
2023-10-26 13:50:13,907 [INFO ] W-9003-model_1 ACCESS_LOG - /169.254.255.130:53288 "POST /invocations HTTP/1.1" 200 1338
2023-10-26 13:50:13,907 [INFO ] W-9003-model_1 TS_METRICS - Requests2XX.Count:1|#Level:Host|#hostname:c449b28b682c,timestamp:null
2023-10-26 13:50:13,907 [INFO ] W-9003-model_1 TS_METRICS - QueueTime.ms:0|#Level:Host|#hostname:c449b28b682c,timestamp:null
2023-10-26 13:50:13,907 [INFO ] W-9003-model_1 TS_METRICS - WorkerThreadTime.ms:0|#Level:Host|#hostname:c449b28b682c,timestamp:null
2023-10-26 13:50:13,907 [INFO ] W-9003-model_1 org.pytorch.serve.wlm.WorkerThread - Backend response time: 1338
2023-10-26 13:50:13,907 [INFO ] W-9003-mod

2023-10-26 13:50:30,885 [INFO ] W-9002-model_1 org.pytorch.serve.wlm.WorkerThread - Backend response time: 1370
2023-10-26 13:50:30,885 [INFO ] W-9002-model_1-stdout MODEL_METRICS - PredictionTime.Milliseconds:1368.76|#ModelName:model,Level:Model|#hostname:c449b28b682c,requestID:9fa11cba-0a3e-4bfb-bca3-3d6037826b2d,timestamp:1698328230
2023-10-26 13:50:30,885 [INFO ] W-9002-model_1 ACCESS_LOG - /169.254.255.130:53288 "POST /invocations HTTP/1.1" 200 1370
2023-10-26 13:50:30,885 [INFO ] W-9002-model_1 TS_METRICS - Requests2XX.Count:1|#Level:Host|#hostname:c449b28b682c,timestamp:null
2023-10-26 13:50:30,885 [INFO ] W-9002-model_1 TS_METRICS - QueueTime.ms:0|#Level:Host|#hostname:c449b28b682c,timestamp:null
2023-10-26 13:50:30,885 [INFO ] W-9002-model_1 TS_METRICS - WorkerThreadTime.ms:0|#Level:Host|#hostname:c449b28b682c,timestamp:null
2023-10-26 13:50:30,885 [INFO ] W-9002-model_1 org.pytorch.serve.wlm.WorkerThread - Backend response time: 1370
2023-10-26 13:50:30,885 [INFO ] W-9002-mod

2023-10-26 13:50:42,428 [INFO ] W-9001-model_1 org.pytorch.serve.wlm.WorkerThread - Backend response time: 1231
2023-10-26 13:50:42,428 [INFO ] W-9001-model_1-stdout MODEL_METRICS - PredictionTime.Milliseconds:1231.01|#ModelName:model,Level:Model|#hostname:c449b28b682c,requestID:aadda497-54a6-451d-9b61-595ba243bebd,timestamp:1698328242
2023-10-26 13:50:42,428 [INFO ] W-9001-model_1 ACCESS_LOG - /169.254.255.130:53288 "POST /invocations HTTP/1.1" 200 1232
2023-10-26 13:50:42,428 [INFO ] W-9001-model_1 org.pytorch.serve.wlm.WorkerThread - Backend response time: 1231
2023-10-26 13:50:42,428 [INFO ] W-9001-model_1-stdout MODEL_METRICS - PredictionTime.Milliseconds:1231.01|#ModelName:model,Level:Model|#hostname:c449b28b682c,requestID:aadda497-54a6-451d-9b61-595ba243bebd,timestamp:1698328242
2023-10-26 13:50:42,428 [INFO ] W-9001-model_1 ACCESS_LOG - /169.254.255.130:53288 "POST /invocations HTTP/1.1" 200 1232
2023-10-26 13:50:42,428 [INFO ] W-9001-model_1 TS_METRICS - Requests2XX.Count:1|#L

2023-10-26 13:50:53,666 [INFO ] W-9000-model_1 org.pytorch.serve.wlm.WorkerThread - Backend response time: 1266
2023-10-26 13:50:53,666 [INFO ] W-9000-model_1-stdout MODEL_METRICS - PredictionTime.Milliseconds:1266.24|#ModelName:model,Level:Model|#hostname:c449b28b682c,requestID:fbccf36f-aafc-4160-bb65-a25faa16ec21,timestamp:1698328253
2023-10-26 13:50:53,667 [INFO ] W-9000-model_1 ACCESS_LOG - /169.254.255.130:53288 "POST /invocations HTTP/1.1" 200 1267
2023-10-26 13:50:53,667 [INFO ] W-9000-model_1 TS_METRICS - Requests2XX.Count:1|#Level:Host|#hostname:c449b28b682c,timestamp:null
2023-10-26 13:50:53,667 [INFO ] W-9000-model_1 TS_METRICS - QueueTime.ms:0|#Level:Host|#hostname:c449b28b682c,timestamp:null
2023-10-26 13:50:53,667 [INFO ] W-9000-model_1 TS_METRICS - WorkerThreadTime.ms:1|#Level:Host|#hostname:c449b28b682c,timestamp:null
2023-10-26 13:50:53,666 [INFO ] W-9000-model_1 org.pytorch.serve.wlm.WorkerThread - Backend response time: 1266
2023-10-26 13:50:53,666 [INFO ] W-9000-mod

2023-10-26 13:51:04,867 [INFO ] W-9002-model_1 org.pytorch.serve.wlm.WorkerThread - Backend response time: 2037
2023-10-26 13:51:04,867 [INFO ] W-9002-model_1-stdout MODEL_METRICS - PredictionTime.Milliseconds:2036.66|#ModelName:model,Level:Model|#hostname:c449b28b682c,requestID:a2d98886-ea7b-40db-988d-dcfcd58a0c2a,timestamp:1698328264
2023-10-26 13:51:04,867 [INFO ] W-9002-model_1 ACCESS_LOG - /169.254.255.130:53288 "POST /invocations HTTP/1.1" 200 2037
2023-10-26 13:51:04,867 [INFO ] W-9002-model_1 TS_METRICS - Requests2XX.Count:1|#Level:Host|#hostname:c449b28b682c,timestamp:null
2023-10-26 13:51:04,867 [INFO ] W-9002-model_1 TS_METRICS - QueueTime.ms:0|#Level:Host|#hostname:c449b28b682c,timestamp:null
2023-10-26 13:51:04,867 [INFO ] W-9002-model_1 TS_METRICS - WorkerThreadTime.ms:0|#Level:Host|#hostname:c449b28b682c,timestamp:null
2023-10-26T13:36:47.914:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=6, BatchStrategy=SINGLE_RECORD
2023-10-26 13:36:45,343 [INFO ] main or

In [20]:
#s3://sdg-project/models/wed_11_10_model2/predictions/text_only_test.csv.out

In [21]:
s3_output_key = os.path.join(prediction_output_path, 'text_only_test.csv.out')

s3_client = boto3.client('s3')
s3_client.download_file(s3_bucket, f'models/{model_name}/predictions/text_only_test.csv.out', local_results_path)

results = pd.read_csv(local_results_path, header=None)
print(results.shape)
results.head()

(498, 17)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,[[-6.7427802085876465,-3.526881,-3.840098,-3.456978,-5.653651,-1.463233,-2.125176,-3.038805,-3.900960,-3.875421,-1.577592,-2.332674,1.505961,-1.963717,0.101850,-3.057724,-1.632241129875183]]
1,[[-11.403116226196289,-2.582178,-0.190443,-8.382274,-7.121809,-4.422887,-6.666330,-6.831847,-4.508859,-6.143005,-6.224436,-4.360418,-5.629896,-4.777306,-4.434818,-7.283071,-4.2934393882751465]]
2,[[-5.668707370758057,-0.109197,-1.049971,-5.457737,-8.217352,0.283350,-4.905684,-2.088828,-2.284471,-4.620093,-2.468168,-1.632369,-1.389243,-3.849088,-2.307996,-3.758041,-0.7262153029441833]]
3,[[-6.214212417602539,-2.695269,-5.482683,-2.395831,-5.196604,-6.662023,-1.982881,0.704692,1.944795,-4.072114,-5.606999,1.248887,-2.856170,-4.680631,-5.028345,-3.648393,-1.4624660015106201]]
4,[[-3.8930087089538574,-2.324404,-4.818657,-2.718109,-1.324449,-6.733837,-4.226032,0.729391,-1.175744,-0.215925,-3.743657,-1.756615,-1.544684,-3.934689,-4.756916,-3.421002,1.5194090604782104]]


In [22]:
results[0] = results[0].str.replace("\\[\\[", "").str.replace("\\]\\]", "").astype(float)
results[16] = results[16].str.replace("\\[\\[", "").str.replace("\\]\\]", "").astype(float)


/tmp/ipykernel_9344/2297004454.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  results[0] = results[0].str.replace("\\[\\[", "").str.replace("\\]\\]", "").astype(float)
/tmp/ipykernel_9344/2297004454.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  results[16] = results[16].str.replace("\\[\\[", "").str.replace("\\]\\]", "").astype(float)


## Inference

In [23]:
# Read the content from the file
with open(local_results_path, 'r') as f:
    lines = f.readlines()

# Process each line to remove the outer brackets and split numbers
processed_lines = [list(map(float, line.strip()[2:-2].split(','))) for line in lines]

# Convert the processed data into a dataframe
results_df = pd.DataFrame(processed_lines)

# print(results_df.shape)
# print(results_df.head())

#processed_lines
results_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,-6.742780,-3.526881,-3.840098,-3.456978,-5.653651,-1.463233,-2.125176,-3.038805,-3.900960,-3.875421,-1.577592,-2.332674,1.505961,-1.963717,0.101850,-3.057724,-1.632241
1,-11.403116,-2.582178,-0.190443,-8.382274,-7.121809,-4.422887,-6.666330,-6.831847,-4.508859,-6.143005,-6.224436,-4.360418,-5.629896,-4.777306,-4.434818,-7.283071,-4.293439
2,-5.668707,-0.109197,-1.049971,-5.457737,-8.217352,0.283350,-4.905684,-2.088828,-2.284471,-4.620093,-2.468168,-1.632369,-1.389243,-3.849088,-2.307996,-3.758041,-0.726215
3,-6.214212,-2.695269,-5.482683,-2.395831,-5.196604,-6.662023,-1.982881,0.704692,1.944795,-4.072114,-5.606999,1.248887,-2.856170,-4.680631,-5.028345,-3.648393,-1.462466
4,-3.893009,-2.324404,-4.818657,-2.718109,-1.324449,-6.733837,-4.226032,0.729391,-1.175744,-0.215925,-3.743657,-1.756615,-1.544684,-3.934689,-4.756916,-3.421002,1.519409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,-4.586705,-9.756909,0.140676,0.927643,-5.052139,-12.516607,-9.832187,-3.127861,-7.149845,-2.605274,-4.543658,-6.353708,-9.456362,-12.584826,-10.690624,-5.809613,-6.874060
494,-7.734004,-2.911306,0.590839,-4.380599,-7.777104,-3.627806,-6.414397,-3.614635,-5.237248,-5.653421,-3.726418,-2.770887,-4.075782,-3.607900,-0.467346,-6.568101,-3.438507
495,-6.219999,-6.429441,-2.165011,1.170356,-4.077498,-10.660872,-9.727581,-3.798720,-4.490309,-1.494787,-5.584704,-5.379607,-7.168537,-9.609615,-6.910304,-7.182022,-4.257317
496,-6.512262,-5.133459,-8.045794,-2.658935,-0.647907,-10.426758,-6.103676,0.025853,-0.088539,-2.700311,-6.036829,-2.192528,-8.765369,-7.485864,-8.334633,-1.796827,-2.810197


In [86]:
# def is_not_float(value):
#     try:
#         float(value)
#         return False
#     except ValueError:
#         return True

# non_float_mask = results_df.applymap(is_not_float)
# non_float_counts = non_float_mask.sum().sum()
# print(f"Total non-float values: {non_float_counts}")

Total non-float values: 0


In [90]:
# def is_float(value):
#     """Check if the value can be converted to float"""
#     try:
#         float(value)
#         return True
#     except:
#         return False

# # Now proceed to convert and check for problematic values
# try:
#     results[0] = results[0].astype(float)
#     results[16] = results[16].astype(float)
# except ValueError as e:
#     print(f"ValueError: {e}")
#     problematic_values_0 = results[~results[0].apply(lambda x: is_float(x))][0]
#     problematic_values_16 = results[~results[16].apply(lambda x: is_float(x))][16]
#     print("Problematic values in column 0:", problematic_values_0.unique())
#     print("Problematic values in column 16:", problematic_values_16.unique())


ValueError: could not convert string to float: '[[-4.270998001098633'
Problematic values in column 0: ['[[-4.270998001098633' '[[-5.848301410675049' '[[-3.6253974437713623'
 '[[-5.789789199829102' '[[-4.9356889724731445' '[[-4.165139198303223'
 '[[-5.735708236694336' '[[-4.352252006530762' '[[-7.999675750732422'
 '[[-6.6009602546691895' '[[-5.733427047729492' '[[-1.9372594356536865'
 '[[-3.4978535175323486' '[[-3.7069549560546875' '[[-3.7345829010009766'
 '[[-1.9820735454559326' '[[-6.994397163391113' '[[-2.0526175498962402'
 '[[-6.503925323486328' '[[-6.751236915588379' '[[-6.309639930725098'
 '[[-6.890380859375' '[[-3.9773223400115967' '[[-5.833578109741211'
 '[[-0.6642332673072815' '[[-4.762781143188477' '[[-4.65572452545166'
 '[[-4.783189296722412' '[[-4.987717628479004' '[[-4.2865519523620605'
 '[[-2.311419725418091' '[[-6.700862884521484' '[[-4.510231971740723'
 '[[-4.784397602081299' '[[-3.649338483810425' '[[-8.45500373840332'
 '[[-4.41664457321167' '[[-7.919408321380615' '[[-4

In [24]:
print(results.dtypes)
# 1. Convert raw scores to probabilities using the sigmoid function.
probabilities = 1 / (1 + np.exp(-results.values))

0     float64
1     float64
2     float64
3     float64
4     float64
5     float64
6     float64
7     float64
8     float64
9     float64
10    float64
11    float64
12    float64
13    float64
14    float64
15    float64
16    float64
dtype: object


In [25]:
probabilities = 1 / (1 + np.exp(-results.values))
probs = pd.DataFrame(probabilities)
probs

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.001178,0.028557,0.021039,0.030561,0.003492,0.187973,0.106674,0.045703,0.019822,0.020324,0.171137,0.088453,0.818462,0.123065,0.525440,0.044885,0.163524
1,0.000011,0.070294,0.452533,0.000229,0.000807,0.011857,0.001271,0.001078,0.010891,0.002144,0.001977,0.012612,0.003576,0.008348,0.011718,0.000687,0.013474
2,0.003440,0.472728,0.259231,0.004245,0.000270,0.570367,0.007350,0.110187,0.092417,0.009756,0.078120,0.163506,0.199529,0.020855,0.090463,0.022798,0.326026
3,0.001997,0.063253,0.004141,0.083491,0.005505,0.001277,0.121012,0.669227,0.874878,0.016756,0.003659,0.777107,0.054363,0.009188,0.006507,0.025372,0.188090
4,0.019977,0.089122,0.008013,0.061913,0.210079,0.001189,0.014400,0.674672,0.235818,0.446227,0.023120,0.147215,0.175855,0.019177,0.008519,0.031646,0.820451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,0.010084,0.000058,0.535111,0.716597,0.006355,0.000004,0.000054,0.041973,0.000784,0.068800,0.010523,0.001737,0.000078,0.000003,0.000023,0.002990,0.001033
494,0.000437,0.051597,0.643558,0.012363,0.000419,0.025887,0.001635,0.026221,0.005287,0.003493,0.023513,0.058918,0.016695,0.026393,0.385245,0.001402,0.031113
495,0.001985,0.001611,0.102937,0.763209,0.016667,0.000023,0.000060,0.021909,0.011093,0.183204,0.003741,0.004588,0.000770,0.000067,0.000996,0.000760,0.013963
496,0.001483,0.005862,0.000320,0.065440,0.343461,0.000030,0.002230,0.506463,0.477880,0.062955,0.002383,0.100423,0.000156,0.000561,0.000240,0.142238,0.056776


In [28]:
print("Shape of true_labels:", true_labels.shape)
print("Shape of binary_predictions:", binary_predictions.shape)

Shape of true_labels: (498, 17)


NameError: name 'binary_predictions' is not defined

In [179]:
test_data

,Text,SDG 1,SDG 2,SDG 3,SDG 4,SDG 5,SDG 6,SDG 7,SDG 8,SDG 9,SDG 10,SDG 11,SDG 12,SDG 13,SDG 14,SDG 15,SDG 16,SDG 17
0,"Shape, Built Enviro Projects the built environ...",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,PEP 5 this course is identified by rmit univer...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Comm Skills for Health Prof this course will e...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,SW Field Education A in this course you will u...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,LSC Work Integrated Learning 2 in this course ...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,Policy analysis for growth prospects in region...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
756,Building an understanding of liveability acros...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
757,Stage 2 - Latrobe Valley Smart Specialisation ...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
758,Balance Victoria: Potential Impacts of a Form...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
from sklearn.metrics import classification_report, accuracy_score, multilabel_confusion_matrix

# # 1. Convert raw scores to probabilities using the sigmoid function.
# probabilities = 1 / (1 + np.exp(-results.values))

# 2. Convert probabilities to binary predictions using a threshold.
threshold = 0.4
binary_predictions = (probabilities > threshold).astype(int)

# 3. Extract true labels from test_data. Assuming test_data has labels in all columns except "Text".
true_labels = test_data.drop(columns="Text").values
print(true_labels.shape)

# Calculate accuracy (this will be a multi-label accuracy).
accuracy = accuracy_score(true_labels, binary_predictions)
print(f"Overall Accuracy: {accuracy:.4f}")

# Detailed classification report.
report = classification_report(true_labels, binary_predictions, target_names=test_data.columns[1:])
print(report)

# Optionally, display the confusion matrix for each SDG (or label).
matrices = multilabel_confusion_matrix(true_labels, binary_predictions)
for idx, matrix in enumerate(matrices):
    print(f"Confusion matrix for SDG {idx+1}:")
    print(matrix)
    print("\n")


(498, 17)
Overall Accuracy: 0.2831
              precision    recall  f1-score   support

       SDG 1       0.50      0.20      0.29        25
       SDG 2       0.80      0.48      0.60        25
       SDG 3       0.82      0.64      0.72       126
       SDG 4       0.68      0.58      0.62       125
       SDG 5       0.60      0.27      0.37        22
       SDG 6       0.68      0.62      0.65        24
       SDG 7       0.75      0.54      0.63        39
       SDG 8       0.57      0.50      0.53        66
       SDG 9       0.50      0.35      0.41        74
      SDG 10       0.49      0.26      0.34        66
      SDG 11       0.71      0.55      0.62        73
      SDG 12       0.62      0.32      0.42        75
      SDG 13       0.47      0.42      0.44        48
      SDG 14       0.60      0.14      0.23        21
      SDG 15       0.59      0.54      0.57        24
      SDG 16       0.65      0.39      0.49        77
      SDG 17       0.43      0.34      0.38   

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
# Write everything to a .txt file
with open(f"Reports/{model_name}_evaluation_report.txt", "w") as f:
    f.write(f"Overall Accuracy: {accuracy:.4f}\n\n")
    f.write("Classification Report:\n")
    f.write(report)
    f.write("\n\n")

    for idx, matrix in enumerate(matrices):
        # Convert the matrix to traditional format
        traditional_matrix = np.array([[matrix[1][1], matrix[1][0]], [matrix[0][1], matrix[0][0]]])
        f.write(f"Confusion matrix for SDG {idx+1}:\n")
        f.write(str(traditional_matrix))
        f.write("\n\n")